In [1]:
import re
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup as bs
import time
import sqlite3
from urllib.parse import quote_plus
import yaml
from datetime import datetime
import os
from KBDownloader import search_swedish_newspapers, fetch_newspaper_data, save_checkpoint, load_checkpoint, retry_on_db_lock, process_and_save_url
from dotenv import load_dotenv
from tqdm import tqdm
import logging
from sqlite3 import OperationalError

# Get today's date
today_date = datetime.today().strftime('%Y-%m-%d')

# Load the YAML configuration file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Load environment variables from .env file
load_dotenv()
kb_key = os.getenv('KB_API_KEY')

# Assign variables from the YAML configuration
venue_list = config['venue_list']
start_year = config['start_year']
years_to_crawl = config['years_to_crawl']
newspaper = config['newspaper']
db_path = config['db_path']
rate_limit = config['rate_limit']
num_composed_blocks = config.get('composed_blocks_context', 1)  # Default to 1 if not specified

# Define the newspaper collection IDs
NEWSPAPER_COLLECTION_IDS = {
    'Dagens nyheter': 'https://libris.kb.se/m5z2w4lz3m2zxpk#it',
    'Svenska Dagbladet': 'https://libris.kb.se/2ldhmx8d4mcrlq9#it',
    'Aftonbladet': 'https://libris.kb.se/dwpgqn5q03ft91j#it',
    'Dagligt Allehanda': 'https://libris.kb.se/9tmqzv3m32xfzcz#it'
}

# Get the correct collection ID for the specified newspaper
collection_id = NEWSPAPER_COLLECTION_IDS.get(newspaper)

#Initialise Database
if not collection_id:
    raise ValueError(f"Invalid newspaper name: {newspaper}")

# Ensure the database file exists
if not os.path.exists(db_path):
    conn = sqlite3.connect(db_path)
    conn.close()

# Create a connection to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create the newspaper_data table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS newspaper_data (
        Date TEXT,
        [Package ID] TEXT,
        Part INTEGER,
        Page INTEGER,
        [ComposedBlock ID] TEXT,
        [ComposedBlock Content] TEXT,
        [Raw API Result] TEXT,
        [Full Prompt] TEXT
    )
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

# Print out all the settings from the YAML configuration file
print("Configuration Settings:")
for key, value in config.items():
    print(f"{key}: {value}")
print(f"Collection ID: {collection_id}")

# Load the venue list
df = pd.read_excel(venue_list)

Configuration Settings:
venue_list: /Users/brandonfarnsworth/Library/Mobile Documents/com~apple~CloudDocs/Post-Phd/Published Texts/Method Article Collecting Trace Data using LLM/Venues_and_Search_Terms.xlsx
start_year: 1848
years_to_crawl: [1848, 1858, 1868, 1878, 1898, 1908]
rate_limit: 0.5
composed_blocks_context: 10
newspaper: Aftonbladet
db_path: Datasets/28.08.24_Stockholm_Touringbot_v1.0.1.db
prompt_filepath: llm_prompt_for_deployment.txt
JSON_schema_path: JSON_Schema.txt
llm_model: gpt-4o-mini-2024-07-18
max_tokens: 1000
Stockholm_Concert_Database_Path: Datasets/All_Concerts_1908_filtered_until_June_30.xlsx
columns_to_compare: ['normalized_date', 'name', 'venue']
column_mapping: {'konsert_datum': 'date', 'konsert_namn': 'name', 'lokal_namn': 'venue', 'arrangör': 'organiser'}
Collection ID: https://libris.kb.se/dwpgqn5q03ft91j#it


In [2]:
# Configure Logging
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    filename='kb_download.log',  # This will create a log file
                    filemode='a')  # 'a' means append (add to existing log file)

# If you want to see logs in the notebook as well, add this:
console = logging.StreamHandler()
console.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

In [3]:
from tqdm import tqdm
import logging
from datetime import datetime
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Load checkpoint if it exists
checkpoint = load_checkpoint()
if checkpoint:
    start_year = checkpoint['year']
    start_half = checkpoint['half']
    start_index = checkpoint['index']
else:
    start_year = years_to_crawl[0]
    start_half = 0
    start_index = 0


total_rows_inserted = 0
for year in years_to_crawl:
    if year < start_year:
        continue
    for half in range(start_half, 2):
        if half == 0:
            from_date = datetime(year, 1, 1)
            to_date = datetime(year, 6, 30)
        else:
            from_date = datetime(year, 7, 1)
            to_date = datetime(year, 12, 31)
        
        logging.info(f"Processing data from {from_date} to {to_date}")
        
        for index in tqdm(range(start_index, len(df)), desc=f"Year {year}, Half {half + 1}"):
            row = df.iloc[index]
            query = row['Lokal']
            
            url_info = {
                'query': query,
                'from_date': from_date.strftime('%Y-%m-%d'),
                'to_date': to_date.strftime('%Y-%m-%d')
            }
            
            success, rows_inserted = process_and_save_url(url_info, config, db_path, kb_key, rate_limit, num_composed_blocks)
            
            if success:
                total_rows_inserted += rows_inserted
                save_checkpoint(year, half, index + 1)
                logging.info(f"Checkpoint saved. Total rows inserted: {total_rows_inserted}")
                if rows_inserted == 0:
                    logging.info(f"No data found for query '{query}'. Moving to next query.")
            else:
                logging.error(f"Failed to process query '{query}'. Moving to next query.")
            
            time.sleep(3)  # Wait for 3 seconds between queries
        
        # Reset start_index for the next half
        start_index = 0
    
    # Reset start_half for the next year
    start_half = 0

logging.info(f"All queries processed for all years. Total rows inserted: {total_rows_inserted}")

2024-08-28 09:31:48,898 - INFO - Processing data from 1848-01-01 00:00:00 to 1848-06-30 00:00:00
2024-08-28 09:31:48,898 - INFO - Processing data from 1848-01-01 00:00:00 to 1848-06-30 00:00:00
Year 1848, Half 1:   0%|          | 0/329 [00:00<?, ?it/s]2024-08-28 09:31:48,916 - INFO - Starting fetch_newspaper_data for query: Konsert, dates: 1848-01-01 to 1848-06-30
2024-08-28 09:31:48,916 - INFO - Starting fetch_newspaper_data for query: Konsert, dates: 1848-01-01 to 1848-06-30
2024-08-28 09:31:49,058 - INFO - Search results received. Hits: 0
2024-08-28 09:31:49,058 - INFO - Search results received. Hits: 0
2024-08-28 09:31:49,059 - INFO - Extracted 0 URLs from search results
2024-08-28 09:31:49,059 - INFO - Extracted 0 URLs from search results
2024-08-28 09:31:49,060 - INFO - Data processing completed. Total rows saved: 0
2024-08-28 09:31:49,060 - INFO - Data processing completed. Total rows saved: 0
2024-08-28 09:31:49,061 - WARNING - Processed URL for query 'Konsert' but no rows were

Checkpoint saved: Year 1848, Half 0, Index 0


Exception: Stopping due to repeated processing failures

In [2]:
# Run if database gets locked on accident
import sqlite3
import os

def recover_database(db_path):
    try:
        conn = sqlite3.connect(db_path)
        conn.execute('VACUUM')
        conn.commit()
        conn.close()
        print(f"Successfully recovered the database: {db_path}")
        return True
    except sqlite3.Error as e:
        print(f"Error recovering the database: {e}")
        return False

def remove_journal_file(db_path):
    journal_path = db_path + '-journal'
    if os.path.exists(journal_path):
        os.remove(journal_path)
        print(f"Removed journal file: {journal_path}")
    else:
        print(f"No journal file found at: {journal_path}")

if __name__ == "__main__":
    if not recover_database(db_path):
        print("Automatic recovery failed. Attempting to remove journal file...")
        remove_journal_file(db_path)
        print("Please try to access the database again.")

Successfully recovered the database: Datasets/23.08_Deployment_Version_1.0.test6.db


# Load SQL Database into Dataframe

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# SQL query to select all columns except 'Raw API Result'
query = """
SELECT Date, [Package ID], Part, Page, [ComposedBlock Content], [Full Prompt]
FROM newspaper_data
"""

# Read the query results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the first few rows of the DataFrame
df.head(10)

# Optional: If you want to save this to a CSV file for further analysis:
# df.to_csv('newspaper_data_summary.csv', index=False)

# STEP 2: Generate Prompts
Note! This is NECESSARY for moving to the next notebook

In [ ]:
# Load system message from the prompt file
with open(config['prompt_filepath'], 'r') as file:
    system_message_content = file.read().strip()

# Load the JSON schema from the file
with open(config['JSON_schema_path'], 'r') as file:
    json_schema = json.load(file)

def generate_full_prompt(row):
    date = row['Date']
    system_message = {"role": "system", "content": system_message_content.replace('{Newspaper_Date}', date)}
    user_content = str(row['ComposedBlock Content'])
    user_message = {"role": "user", "content": user_content}
    package_id = row['Package ID']
    part = row['Part']
    page = row['Page']

    # Get the row index to use as a sequential index
    row_index = row.name + 1

    custom_id = f"{package_id}-{part}-{page}-{row_index}"

    # Load the JSON schema from the file
    with open(config['JSON_schema_path'], 'r') as file:
        json_schema = json.load(file)

    # Prepare the full prompt JSON
    full_prompt = {
        "custom_id": custom_id,
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": config['llm_model'],
            "messages": [system_message, user_message],
            "max_tokens": config['max_tokens'],
            "response_format": {
                "type": "json_schema",
                "json_schema": {
                    "name": "response_data",
                    "strict": True,
                    "schema": json_schema
                }
            }
        }
    }

    return json.dumps(full_prompt)

# Generate the Full Prompt column
df['Full Prompt'] = df.apply(generate_full_prompt, axis=1)

df.head()

# Commit prompts to database

In [ ]:
import sqlite3
from sqlalchemy import create_engine

# Establish a connection to the database
engine = create_engine(f'sqlite:///{db_path}', echo=False)

# Update the existing table with the DataFrame containing the new 'Full Prompt' column
df.to_sql('newspaper_data', engine, if_exists='replace', index=False)

# Commit the changes and close the connection
conn = sqlite3.connect(db_path)
conn.commit()
conn.close()

print("Updated data committed to the 'newspaper_data' table. Rows: {}".format(len(df)))